In [7]:
import pandas as pd
import numpy as np

In [155]:
inFea = pd.read_csv("../Data/features_30_sec.csv")
inFea
features_in = inFea[["filename","label"]]

In [161]:
features_in

,filename,label
0,blues.00000.wav,blues
1,blues.00001.wav,blues
2,blues.00002.wav,blues
3,blues.00003.wav,blues
4,blues.00004.wav,blues
...,...,...
995,rock.00095.wav,rock
996,rock.00096.wav,rock
997,rock.00097.wav,rock
998,rock.00098.wav,rock


In [47]:
!pip install -v "librosa==0.9"

Using pip 24.2 from /opt/anaconda3/lib/python3.12/site-packages/pip (python 3.12)
  Obtaining dependency information for librosa==0.9 from https://files.pythonhosted.org/packages/5a/90/61c239b4bf9aee2ac16dcf1a1f05d5f112ac0ad301f06feacb42fa6834aa/librosa-0.9.0-py3-none-any.whl.metadata
  Using cached librosa-0.9.0-py3-none-any.whl.metadata (6.9 kB)
Using cached librosa-0.9.0-py3-none-any.whl (211 kB)
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.2.post1
    Uninstalling librosa-0.10.2.post1:
      Removing file or directory /opt/anaconda3/lib/python3.12/site-packages/librosa-0.10.2.post1.dist-info/
      Removing file or directory /opt/anaconda3/lib/python3.12/site-packages/librosa/__init__.py
      Removing file or directory /opt/anaconda3/lib/python3.12/site-packages/librosa/__init__.pyi
      Removing file or directory /opt/anaconda3/lib/python3.12/site-packages/librosa/__pycache__/__init__.cpython-312.pyc
      Removing file or directory /opt/anaconda

In [49]:
def load(filepath):

    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        try:
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                    'category', categories=SUBSETS, ordered=True)
        except (ValueError, TypeError):
            # the categories and ordered arguments were removed in pandas 0.25
            tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                     pd.CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks


In [165]:
#!/usr/bin/env python3

# FMA: A Dataset For Music Analysis
# Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst, Xavier Bresson, EPFL LTS2.

# All features are extracted using [librosa](https://github.com/librosa/librosa).
# Alternatives:
# * [Essentia](http://essentia.upf.edu) (C++ with Python bindings)
# * [MARSYAS](https://github.com/marsyas/marsyas) (C++ with Python bindings)
# * [RP extract](http://www.ifs.tuwien.ac.at/mir/downloads.html) (Matlab, Java, Python)
# * [jMIR jAudio](http://jmir.sourceforge.net) (Java)
# * [MIRtoolbox](https://www.jyu.fi/hum/laitokset/musiikki/en/research/coe/materials/mirtoolbox) (Matlab)

import os
import multiprocessing
import warnings
import numpy as np
from scipy import stats
import pandas as pd
import librosa
from tqdm import tqdm


def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)
    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def compute_features(audioName,genreName):

    features = pd.Series(index=columns(), dtype=np.float32)
    
    # Catch warnings as exceptions (audioread leaks file descriptors).
    warnings.filterwarnings('error', module='librosa')

    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

    try:
        filepath = "../Data/genres_original/"+genreName+'/'+audioName
        x, sr = librosa.load(filepath, sr=None, mono=True)  # kaiser_fast

        f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
        feature_stats('zcr', f)

        cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
        assert cqt.shape[0] == 7 * 12
        assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

        f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cqt', f)
        f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cens', f)
        f = librosa.feature.tonnetz(chroma=f)
        feature_stats('tonnetz', f)

        del cqt
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x

        f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
        feature_stats('chroma_stft', f)

        f = librosa.feature.rms(S=stft)
        feature_stats('rmse', f)

        f = librosa.feature.spectral_centroid(S=stft)
        feature_stats('spectral_centroid', f)
        f = librosa.feature.spectral_bandwidth(S=stft)
        feature_stats('spectral_bandwidth', f)
        f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
        feature_stats('spectral_contrast', f)
        f = librosa.feature.spectral_rolloff(S=stft)
        feature_stats('spectral_rolloff', f)

        mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        del stft
        f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
        feature_stats('mfcc', f)

    except Exception as e:
        print('{}'.format( repr(e)))

    return features



 

In [202]:
feature_val = features_in.apply(lambda row: compute_features(row['filename'], row['label']), axis=1)

/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_33017/1272142355.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.08304482]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  features[name, 'mean'] = np.mean(values, axis=1)
/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_33017/1272142355.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.05603981]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  features[name, 'mean'] = np.mean(values, axis=1)
/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_33017/1272142355.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.07629121]' has dtype incompatible with float32, pleas

NoBackendError()


/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_33017/1272142355.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.11474046]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  features[name, 'mean'] = np.mean(values, axis=1)
/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_33017/1272142355.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.06185878]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  features[name, 'mean'] = np.mean(values, axis=1)
/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_33017/1272142355.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.09960031]' has dtype incompatible with float32, pleas

In [208]:
newFeatureCols = [("track_id",'',''),('genre','','')]+feature_val.columns.to_list()

In [211]:
features_all=pd.concat([features_in,feature_val],axis=1)

In [213]:
features_all.columns = pd.MultiIndex.from_tuples(newFeatureCols)

In [215]:
features_all


track_id  genre chroma_cens                                \
                               kurtosis                                 
                                     01        02        03        04   
0    blues.00000.wav  blues   -0.078091 -0.290040  0.142337  0.046511   
1    blues.00001.wav  blues    0.497728 -0.156311  0.986335  0.067993   
2    blues.00002.wav  blues   -0.532496 -0.776484  1.007275  0.825708   
3    blues.00003.wav  blues    3.513676  0.430777  0.822450 -1.208157   
4    blues.00004.wav  blues    1.143850  0.123680 -0.372205 -0.437249   
..               ...    ...         ...       ...       ...       ...   
995   rock.00095.wav   rock    0.464216 -0.515369 -0.988768 -0.729247   
996   rock.00096.wav   rock   -0.859656  1.034301 -0.437038 -0.836010   
997   rock.00097.wav   rock    0.262118 -0.432864  0.424227 -0.359552   
998   rock.00098.wav   rock    0.515022 -1.052786  3.600510  0.012773   
999   rock.00099.wav   rock   -0.584398 -1.296309 -0.860861 -0.751995   

                                             ...   tonnetz            \
                                             ...       std             
           05        06        07        08  ...        04        05   
0   -1.092855 -0.786504 -0.835958 -0.446122  ...  0.138799  0.026220   
1   -0.790453  3.687446 -1.138991 -0.923531  ...  0.179517  0.029268   
2    0.276671 -0.682883  1.406657  0.022634  ...  0.073724  0.033821   
3   -0.965539 -1.065736 -0.546926 -1.031764  ...  0.132739  0.060522   
4   -0.086288  0.243630 -0.011765 -0.096848  ...  0.089078  0.032872   
..        ...       ...       ...       ...  ...       ...       ...   
995 -0.215835 -0.199535 -1.169909 -0.421062  ...  0.133570  0.056552   
996 -0.170238 -0.948016 -1.013720 -0.831747  ...  0.084923  0.074198   
997 -0.628410 -0.872857 -0.822605 -0.701298  ...  0.095067  0.031494   
998 -0.701663  0.112954 -0.750285 -0.647902  ...  0.169529  0.050952   
999 -1.253281 -1.599412 -1.375497  0.329813  ...  0.233966  0.084716   

                     zcr                                                    \
                kurtosis       max      mean    median       min      skew   
           06         01        01        01        01        01        01   
0    0.037989   3.902308  0.242676  0.083045  0.078125  0.033203  1.427675   
1    0.041067   9.201668  0.323730  0.056040  0.047852  0.006348  2.363820   
2    0.028017  37.734068  0.440918  0.076291  0.073730  0.012695  4.185553   
3    0.034751   2.021524  0.145020  0.033309  0.030762  0.006348  1.073028   
4    0.019889   5.706599  0.366211  0.101461  0.092773  0.039551  1.938532   
..        ...        ...       ...       ...       ...       ...       ...   
995  0.041105   4.357486  0.318848  0.089227  0.084473  0.021973  1.406124   
996  0.026935   0.690220  0.195312  0.097664  0.098145  0.011719 -0.306379   
997  0.034359   2.288512  0.314453  0.121823  0.122559  0.026367  0.811464   
998  0.069180   2.488822  0.193848  0.048724  0.042969  0.008789  1.362758   
999  0.076957  -0.054174  0.237305  0.076305  0.067871  0.007324  0.806931   

               
          std  
           01  
0    0.027694  
1    0.038046  
2    0.031731  
3    0.020561  
4    0.044205  
..        ...  
995  0.036121  
996  0.029454  
997  0.041192  
998  0.028417  
999  0.046597  

[1000 rows x 520 columns]

In [218]:
features_all.to_csv("custom_more_data.csv")

In [849]:
features_all_one_hot_df = features_all['genre'].apply(lambda x: pd.Series({val: 1 for val in x})).fillna(0)

In [220]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier

In [319]:
model = RandomForestClassifier(n_estimators=100)

In [321]:
X = features_all.drop(["track_id","genre"],axis=1)
y = features_all["genre"]

/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_33017/1145221567.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = features_all.drop(["track_id","genre"],axis=1)


In [336]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [338]:
model.fit(x_train,y_train)

RandomForestClassifier()

In [340]:
y_pred=model.predict(x_test)

In [342]:
accuracy = accuracy_score(y_test,y_pred)

In [344]:
accuracy

0.79

In [362]:
itend = pd.read_csv('itend.csv',index_col=[0],header=[0, 1,2])

In [364]:
itend

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
0            -0.140905 -0.213307 -0.416046 -1.174401 -1.175695  0.382804   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
0           0.045361 -0.807864 -0.578678 -0.104263  ...  0.150906  0.047585   

feature                     zcr                                              \
statistics             kurtosis       max     mean    median  min      skew   
number            06         01        01       01        01   01        01   
0           0.058541  39.566994  0.575195  0.05988  0.056152  0.0  4.427992   

feature               
statistics       std  
number            01  
0           0.039989  

[1 rows x 518 columns]

In [368]:
model.predict(itend)

array(['hiphop'], dtype=object)

In [426]:
features = load("../../Genre Prediction/Data/Original Data/fma_metadata/features.csv")

In [428]:
features

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
20           -0.193837 -0.198527  0.201546  0.258556  0.775204  0.084794   
...                ...       ...       ...       ...       ...       ...   
155316       -0.490129  0.463834  2.321970 -0.084352  1.662914  2.115189   
155317       -0.461559 -0.229601 -0.496632 -0.422033  0.130612 -0.263825   
155318        0.552473 -0.110498 -0.532014  0.263131 -0.224011 -0.530972   
155319       -0.176901  0.187208 -0.050664  0.368843  0.066005 -0.857354   
155320        0.489665  1.862421  0.854461 -0.103666 -0.249835  0.360283   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
20         -0.289294 -0.816410  0.043851 -0.804761  ...  0.095003  0.022492   
...              ...       ...       ...       ...  ...       ...       ...   
155316     -0.237794  5.695442  0.830353  1.951819  ...  0.128410  0.022547   
155317     -0.628103 -0.082687 -0.229483 -0.492753  ...  0.132964  0.023548   
155318      1.713526  1.418444  1.325197  0.120333  ...  0.108324  0.017540   
155319     -0.780860  0.626281 -0.630938 -0.787229  ...  0.088311  0.018328   
155320     -0.366701  0.033578 -0.834606 -1.154845  ...  0.091421  0.020312   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
track_id                                                                  
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.000000   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.000000   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.000000   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.000000   
20          0.021355  16.669037  0.469727  0.047225  0.040039  0.000977   
...              ...        ...       ...       ...       ...       ...   
155316      0.019816   4.448255  0.172852  0.028773  0.028320  0.003906   
155317      0.026527   3.270612  0.196289  0.031116  0.027832  0.002441   
155318      0.020471   2.356727  0.212891  0.038450  0.037109  0.003418   
155319      0.017936   6.188604  0.167480  0.041480  0.038086  0.004883   
155320      0.016794  21.756050  0.845215  0.075141  0.044434  0.004395   

feature                         
statistics      skew       std  
number            01        01  
track_id                        
2           2.089872  0.061448  
3           1.716724  0.069330  
5           2.193303  0.044861  
10          3.542325  0.040800  
20          3.189831  0.030993  
...              ...       ...  
155316      0.955388  0.012385  
155317      1.283060  0.019059  
155318      0.828569  0.017904  
155319      1.818740  0.020133  
155320      4.687204  0.137205  

[106574 rows x 518 columns]

In [380]:
import ast
tracks = load("../../Genre Prediction/Data/Original Data/fma_metadata/tracks.csv")

In [386]:
tracks=tracks['track']

In [390]:
tracks_genre = tracks.reset_index()[['track_id','genre_top']]

In [392]:
tracks_genre

,track_id,genre_top
0,2,Hip-Hop
1,3,Hip-Hop
2,5,Hip-Hop
3,10,Pop
4,20,NaN
...,...,...
106569,155316,Rock
106570,155317,Rock
106571,155318,Rock
106572,155319,Rock


In [400]:
col_fea = features.columns

In [430]:
features = features.droplevel(0,axis=1)
features = features.droplevel(0,axis=1)

In [434]:
features.reset_index(inplace=True)

In [436]:
features_genre = pd.merge(tracks_genre,features,on="track_id",how="inner")

In [462]:
features_genre.columns = pd.MultiIndex.from_tuples([("track_id",'',''),("genre",'','')]+col_fea.to_list()[1:])

In [464]:
features_genre.to_csv("feature_gerne_clean.csv")

In [466]:
features_genre

track_id    genre chroma_cens                                          \
                            kurtosis                                           
                                  01        02        03        04        05   
0             2  Hip-Hop    7.180653  5.230309  0.249321  1.347620  1.482478   
1             3  Hip-Hop    1.888963  0.760539  0.345297  2.295201  1.654031   
2             5  Hip-Hop    0.527563 -0.077654 -0.279610  0.685883  1.937570   
3            10      Pop    3.702245 -0.291193  2.196742 -0.234449  1.367364   
4            20      NaN   -0.193837 -0.198527  0.201546  0.258556  0.775204   
...         ...      ...         ...       ...       ...       ...       ...   
106569   155316     Rock   -0.490129  0.463834  2.321970 -0.084352  1.662914   
106570   155317     Rock   -0.461559 -0.229601 -0.496632 -0.422033  0.130612   
106571   155318     Rock    0.552473 -0.110498 -0.532014  0.263131 -0.224011   
106572   155319     Rock   -0.176901  0.187208 -0.050664  0.368843  0.066005   
106573   155320      NaN    0.489665  1.862421  0.854461 -0.103666 -0.249835   

                                      ...   tonnetz                      \
                                      ...       std                       
              06        07        08  ...        04        05        06   
0       0.531371  1.481593  2.691455  ...  0.054125  0.012226  0.012111   
1       0.067592  1.366848  1.054094  ...  0.063831  0.014212  0.017740   
2       0.880839 -0.923192 -0.927232  ...  0.040730  0.012691  0.014759   
3       0.998411  1.770694  1.604566  ...  0.074358  0.017952  0.013921   
4       0.084794 -0.289294 -0.816410  ...  0.095003  0.022492  0.021355   
...          ...       ...       ...  ...       ...       ...       ...   
106569  2.115189 -0.237794  5.695442  ...  0.128410  0.022547  0.019816   
106570 -0.263825 -0.628103 -0.082687  ...  0.132964  0.023548  0.026527   
106571 -0.530972  1.713526  1.418444  ...  0.108324  0.017540  0.020471   
106572 -0.857354 -0.780860  0.626281  ...  0.088311  0.018328  0.017936   
106573  0.360283 -0.366701  0.033578  ...  0.091421  0.020312  0.016794   

              zcr                                                              
         kurtosis       max      mean    median       min      skew       std  
               01        01        01        01        01        01        01  
0        5.758890  0.459473  0.085629  0.071289  0.000000  2.089872  0.061448  
1        2.824694  0.466309  0.084578  0.063965  0.000000  1.716724  0.069330  
2        6.808415  0.375000  0.053114  0.041504  0.000000  2.193303  0.044861  
3       21.434212  0.452148  0.077515  0.071777  0.000000  3.542325  0.040800  
4       16.669037  0.469727  0.047225  0.040039  0.000977  3.189831  0.030993  
...           ...       ...       ...       ...       ...       ...       ...  
106569   4.448255  0.172852  0.028773  0.028320  0.003906  0.955388  0.012385  
106570   3.270612  0.196289  0.031116  0.027832  0.002441  1.283060  0.019059  
106571   2.356727  0.212891  0.038450  0.037109  0.003418  0.828569  0.017904  
106572   6.188604  0.167480  0.041480  0.038086  0.004883  1.818740  0.020133  
106573  21.756050  0.845215  0.075141  0.044434  0.004395  4.687204  0.137205  

[106574 rows x 520 columns]

In [460]:
features_all

track_id  genre chroma_cens                                \
                               kurtosis                                 
                                     01        02        03        04   
0    blues.00000.wav  blues   -0.078091 -0.290040  0.142337  0.046511   
1    blues.00001.wav  blues    0.497728 -0.156311  0.986335  0.067993   
2    blues.00002.wav  blues   -0.532496 -0.776484  1.007275  0.825708   
3    blues.00003.wav  blues    3.513676  0.430777  0.822450 -1.208157   
4    blues.00004.wav  blues    1.143850  0.123680 -0.372205 -0.437249   
..               ...    ...         ...       ...       ...       ...   
995   rock.00095.wav   rock    0.464216 -0.515369 -0.988768 -0.729247   
996   rock.00096.wav   rock   -0.859656  1.034301 -0.437038 -0.836010   
997   rock.00097.wav   rock    0.262118 -0.432864  0.424227 -0.359552   
998   rock.00098.wav   rock    0.515022 -1.052786  3.600510  0.012773   
999   rock.00099.wav   rock   -0.584398 -1.296309 -0.860861 -0.751995   

                                             ...   tonnetz            \
                                             ...       std             
           05        06        07        08  ...        04        05   
0   -1.092855 -0.786504 -0.835958 -0.446122  ...  0.138799  0.026220   
1   -0.790453  3.687446 -1.138991 -0.923531  ...  0.179517  0.029268   
2    0.276671 -0.682883  1.406657  0.022634  ...  0.073724  0.033821   
3   -0.965539 -1.065736 -0.546926 -1.031764  ...  0.132739  0.060522   
4   -0.086288  0.243630 -0.011765 -0.096848  ...  0.089078  0.032872   
..        ...       ...       ...       ...  ...       ...       ...   
995 -0.215835 -0.199535 -1.169909 -0.421062  ...  0.133570  0.056552   
996 -0.170238 -0.948016 -1.013720 -0.831747  ...  0.084923  0.074198   
997 -0.628410 -0.872857 -0.822605 -0.701298  ...  0.095067  0.031494   
998 -0.701663  0.112954 -0.750285 -0.647902  ...  0.169529  0.050952   
999 -1.253281 -1.599412 -1.375497  0.329813  ...  0.233966  0.084716   

                     zcr                                                    \
                kurtosis       max      mean    median       min      skew   
           06         01        01        01        01        01        01   
0    0.037989   3.902308  0.242676  0.083045  0.078125  0.033203  1.427675   
1    0.041067   9.201668  0.323730  0.056040  0.047852  0.006348  2.363820   
2    0.028017  37.734068  0.440918  0.076291  0.073730  0.012695  4.185553   
3    0.034751   2.021524  0.145020  0.033309  0.030762  0.006348  1.073028   
4    0.019889   5.706599  0.366211  0.101461  0.092773  0.039551  1.938532   
..        ...        ...       ...       ...       ...       ...       ...   
995  0.041105   4.357486  0.318848  0.089227  0.084473  0.021973  1.406124   
996  0.026935   0.690220  0.195312  0.097664  0.098145  0.011719 -0.306379   
997  0.034359   2.288512  0.314453  0.121823  0.122559  0.026367  0.811464   
998  0.069180   2.488822  0.193848  0.048724  0.042969  0.008789  1.362758   
999  0.076957  -0.054174  0.237305  0.076305  0.067871  0.007324  0.806931   

               
          std  
           01  
0    0.027694  
1    0.038046  
2    0.031731  
3    0.020561  
4    0.044205  
..        ...  
995  0.036121  
996  0.029454  
997  0.041192  
998  0.028417  
999  0.046597  

[1000 rows x 520 columns]

In [471]:
all_data = pd.concat([features_genre,features_all])

track_id    genre chroma_cens                                \
                                kurtosis                                 
                                      01        02        03        04   
0                 2  Hip-Hop    7.180653  5.230309  0.249321  1.347620   
1                 3  Hip-Hop    1.888963  0.760539  0.345297  2.295201   
2                 5  Hip-Hop    0.527563 -0.077654 -0.279610  0.685883   
3                10      Pop    3.702245 -0.291193  2.196742 -0.234449   
4                20      NaN   -0.193837 -0.198527  0.201546  0.258556   
..              ...      ...         ...       ...       ...       ...   
995  rock.00095.wav     rock    0.464216 -0.515369 -0.988768 -0.729247   
996  rock.00096.wav     rock   -0.859656  1.034301 -0.437038 -0.836010   
997  rock.00097.wav     rock    0.262118 -0.432864  0.424227 -0.359552   
998  rock.00098.wav     rock    0.515022 -1.052786  3.600510  0.012773   
999  rock.00099.wav     rock   -0.584398 -1.296309 -0.860861 -0.751995   

                                             ...   tonnetz            \
                                             ...       std             
           05        06        07        08  ...        04        05   
0    1.482478  0.531371  1.481593  2.691455  ...  0.054125  0.012226   
1    1.654031  0.067592  1.366848  1.054094  ...  0.063831  0.014212   
2    1.937570  0.880839 -0.923192 -0.927232  ...  0.040730  0.012691   
3    1.367364  0.998411  1.770694  1.604566  ...  0.074358  0.017952   
4    0.775204  0.084794 -0.289294 -0.816410  ...  0.095003  0.022492   
..        ...       ...       ...       ...  ...       ...       ...   
995 -0.215835 -0.199535 -1.169909 -0.421062  ...  0.133570  0.056552   
996 -0.170238 -0.948016 -1.013720 -0.831747  ...  0.084923  0.074198   
997 -0.628410 -0.872857 -0.822605 -0.701298  ...  0.095067  0.031494   
998 -0.701663  0.112954 -0.750285 -0.647902  ...  0.169529  0.050952   
999 -1.253281 -1.599412 -1.375497  0.329813  ...  0.233966  0.084716   

                     zcr                                                    \
                kurtosis       max      mean    median       min      skew   
           06         01        01        01        01        01        01   
0    0.012111   5.758890  0.459473  0.085629  0.071289  0.000000  2.089872   
1    0.017740   2.824694  0.466309  0.084578  0.063965  0.000000  1.716724   
2    0.014759   6.808415  0.375000  0.053114  0.041504  0.000000  2.193303   
3    0.013921  21.434212  0.452148  0.077515  0.071777  0.000000  3.542325   
4    0.021355  16.669037  0.469727  0.047225  0.040039  0.000977  3.189831   
..        ...        ...       ...       ...       ...       ...       ...   
995  0.041105   4.357486  0.318848  0.089227  0.084473  0.021973  1.406124   
996  0.026935   0.690220  0.195312  0.097664  0.098145  0.011719 -0.306379   
997  0.034359   2.288512  0.314453  0.121823  0.122559  0.026367  0.811464   
998  0.069180   2.488822  0.193848  0.048724  0.042969  0.008789  1.362758   
999  0.076957  -0.054174  0.237305  0.076305  0.067871  0.007324  0.806931   

               
          std  
           01  
0    0.061448  
1    0.069330  
2    0.044861  
3    0.040800  
4    0.030993  
..        ...  
995  0.036121  
996  0.029454  
997  0.041192  
998  0.028417  
999  0.046597  

[107574 rows x 520 columns]

In [477]:
X = all_data.dropna().drop(["track_id","genre"],axis=1)
y = all_data.dropna()["genre"]

/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_33017/2566396673.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = all_data.dropna().drop(["track_id","genre"],axis=1)


In [780]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [828]:
new_model = RandomForestClassifier(n_estimators=1000)

In [784]:
new_model.fit(x_train,y_train)

RandomForestClassifier()

In [785]:
new_y_pred=new_model.predict(x_test)

In [786]:
accuracy_new = accuracy_score(y_test,new_y_pred)

In [646]:
from imblearn.under_sampling import RandomUnderSampler

undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

NameError: name 'X_train' is not defined

In [787]:
accuracy_new

0.6228590250329381

In [788]:
from imblearn.under_sampling import RandomUnderSampler

undersampler = RandomUnderSampler()
X_resampled, y_resampled = undersampler.fit_resample(X, y)

In [789]:
y_resampled.value_counts()

genre
Blues                  24
Classical              24
reggae                 24
pop                    24
metal                  24
jazz                   24
hiphop                 24
disco                  24
country                24
classical              24
blues                  24
Spoken                 24
Soul-RnB               24
Rock                   24
Pop                    24
Old-Time / Historic    24
Jazz                   24
International          24
Instrumental           24
Hip-Hop                24
Folk                   24
Experimental           24
Electronic             24
Easy Listening         24
Country                24
rock                   24
Name: count, dtype: int64

In [811]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3)

In [830]:
new_model.fit(X_resampled,y_resampled)

RandomForestClassifier(n_estimators=1000)

In [832]:
new_y= new_model.predict(x_test)

In [834]:
accuracy_newst = accuracy_score(y_test,new_y)

In [837]:
accuracy_newst

1.0

In [839]:
new_y == y_test

41569    True
1779     True
34427    True
468      True
753      True
         ... 
84135    True
28464    True
84791    True
9711     True
1806     True
Name: genre, Length: 188, dtype: bool

In [841]:
new_model.predict(itend)

array(['hiphop'], dtype=object)

In [853]:
features_all_one_hot_df.columns

Index(['b', 'l', 'u', 'e', 's', 'c', 'a', 'i', 'o', 'n', 't', 'r', 'y', 'd',
       'h', 'p', 'j', 'z', 'm', 'g', 'k'],
      dtype='object')